# <h1 style="color: purple;">Step 1 - Scraping</h1>

### NASA Mars News

In [1]:
# Import Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.common.exceptions import ElementNotVisibleException

In [2]:
# Scrape the NASA Mars News Site here (https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest)
# and collect the LATEST News TITLE and PARAGRAPH Text. Assign the text to variables that you can reference later.
# Example:

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://mars.nasa.gov/news/'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Get the list of news 
ul_list_news = soup.find('ul', class_='item_list')
# Get the last news
li_last_news = ul_list_news.find_all('li')[0]
# Get the link for the last news.
link_latest_news = li_last_news.find('a')['href']
print(link_latest_news)

/news/8503/robotic-toolkit-added-to-nasas-mars-2020-rover/


In [3]:
# Go to the news page
url = url.replace("/news","",1) + link_latest_news
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Get the title
last_title = soup.find('h1', class_='article_title')
print(last_title.text)

# Get the paragraph list
lst_p = soup.find('div', class_='wysiwyg_content')
# Get the first paragraph
last_p = lst_p.find_all('p')[0]
print(last_p.text)


Robotic Toolkit Added to NASA's Mars 2020 Rover

The bit carousel — a mechanism that will play a key role in the acquisition, containment and eventual return to Earth of humanity's first samples from another planet — has been incorporated into NASA's Mars 2020 rover.


### JPL Mars Space Images - Featured Image

In [5]:
# Visit the url for JPL Featured Space Image here ==> https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars.
# Go to the news page
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Get the figure ID.
carousel_item = soup.find('article', class_="carousel_item")
footer = carousel_item.find('footer')
id_fig = footer.find('a')['data-link']
id_fig = id_fig.split("=")[1]

# Use splinter to navigate the site and find the image url for the current Featured 
# Mars Image and assign the url string to a variable called featured_image_url.
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/images/largesize/' + id_fig + '_hires.jpg'

print("The URL is {}".format(featured_image_url))

The URL is https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18358_hires.jpg


### Mars Weather

In [7]:
# Visit the Mars Weather twitter account here (https://twitter.com/marswxreport?lang=en) and scrape the latest 
# Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


# sidebar = soup.find('ul', class_='nav-list')
# categories = sidebar.find_all('li')


ol = soup.find('ol', class_='stream-items js-navigable-stream')
li = ol.find_all('li')[0]
div = li.find('div', class_='js-tweet-text-container')
mars_weather = div.find('p').text
print (mars_weather)


InSight sol 258 (2019-08-18) low -100.0ºC (-148.1ºF) high -26.2ºC (-15.2ºF)
winds from the SSE at 5.3 m/s (11.9 mph) gusting to 16.8 m/s (37.6 mph)
pressure at 7.60 hPapic.twitter.com/5nCVjcsmlZ


### Mars Facts

In [8]:
# Visit the Mars Facts webpage here (https://space-facts.com/mars/) and use Pandas to scrape 
# the table containing facts about the planet including Diameter, Mass, etc.
url = "https://space-facts.com/mars/"
tables = pd.read_html(url)
df = tables[1]
df.rename(columns={0:'info',1:'value'}, inplace=True)
df.set_index('info', inplace=True)

df.head(20)


,value
info,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
# Use Pandas to convert the data to a HTML table string.
html_table = df.to_html()
html_table = html_table.replace('\n', '')
html_table


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>info</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

### Mars Hemispheres

In [10]:
# Visit the USGS Astrogeology site here (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)
# to obtain high resolution images for each of Mar's hemispheres.
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

url_lst =[]
lst = soup.find('div', class_='collapsible results')
lst_a = lst.find_all('div', class_="item")

for item in lst_a:
    lala = item.find('div', class_='description')
    url_lst.append('https://astrogeology.usgs.gov' +  lala.find('a')['href'])


In [11]:
# You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
hemisphere_image_urls =[]
for url_fig in url_lst:
    browser.visit(url_fig)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing
    # the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
    html_block = soup.find('div', class_='downloads')
    img_url = 'https://astrogeology.usgs.gov' + html_block.find('img')['src']
    title = soup.title.text
    title = title.split("|")[0]
    title = title.replace("Enhanced","")

    # Append the dictionary with the image url string and the hemisphere title to a list. 
    # This list will contain one dictionary for each hemisphere.
    hemisphere_image_urls.append({"title":title, "img_url": img_url})     
    
print (hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere  ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/dfaf3849e74bf973b59eb50dab52b583_cerberus_enhanced.tif_thumb.png'}, {'title': 'Schiaparelli Hemisphere  ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7677c0a006b83871b5a2f66985ab5857_schiaparelli_enhanced.tif_thumb.png'}, {'title': 'Syrtis Major Hemisphere  ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/aae41197e40d6d4f3ea557f8cfe51d15_syrtis_major_enhanced.tif_thumb.png'}, {'title': 'Valles Marineris Hemisphere  ', 'img_url': 'https://astrogeology.usgs.gov/cache/images/04085d99ec3713883a9a57f42be9c725_valles_marineris_enhanced.tif_thumb.png'}]
